# Tate Project in Python
Tasks:
1. Which artist in this data set lived the longest?
2. Who are the top 10 artists by the number of artworks?
3. Which artist is created the most artwork by total surface area?
4. Did any artists have artwork acquired during their lifetime?
5. Please review the quality of the data, and present any issues
6. Please group the artworks into as many clusters as you feel is appropriate, using attributes from both the artist and artworks tables, and assign each artwork to this new cluster.

### Prepare data
Data is cleaned in preparation for answering above questions

##### Import libraries and read data from db

In [18]:
import numpy as np 
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
from skimpy import clean_columns

path = "C:/Users/katba/Data analysis learning/Portfolio/tate_project/data/"
database = path + 'artists.db'

In [13]:
conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,artists,artists,2,"CREATE TABLE ""artists"" (\n""Artist ID"" INTEGER,..."
1,table,artworks,artworks,159,"CREATE TABLE ""artworks"" (\n""Artwork ID"" INTEGE..."


In [16]:
artists = pd.read_sql("""SELECT *
                        FROM artists;""", conn)

artists

,Artist ID,Name,Nationality,Gender,Birth Year,Death Year
0,1,Robert Arneson,American,Male,1930.0,1992.0
1,2,Doroteo Arnaiz,Spanish,Male,1936.0,NaN
2,3,Bill Arnold,American,Male,1941.0,NaN
3,4,Charles Arnoldi,American,Male,1946.0,NaN
4,5,Per Arnoldi,Danish,Male,1941.0,NaN
...,...,...,...,...,...,...
15086,67452,Liu Jianhua,Chinese,Male,1962.0,NaN
15087,67453,Leng Lin,Chinese,Male,1965.0,NaN
15088,67652,Ellie Nagler,None,None,NaN,NaN
15089,67694,Glenn Williams,None,Male,NaN,NaN


In [17]:
artworks = pd.read_sql("""SELECT *
                        FROM artworks;""", conn)

artworks

,Artwork ID,Title,Artist ID,Name,Date,Medium,Dimensions,Acquisition Date,Credit,Catalogue,...,Classification,Object Number,Diameter (cm),Circumference (cm),Height (cm),Length (cm),Width (cm),Depth (cm),Weight (kg),Duration (s)
0,2,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",6210,Otto Wagner,1896,Ink and cut-and-pasted painted pages on paper,"19 1/8 x 66 1/2"" (48.6 x 168.9 cm)",1996-04-09,Fractional and promised gift of Jo Carole and ...,Y,...,Architecture,885.1996,NaN,NaN,48.6000,NaN,168.9000,NaN,NaN,NaN
1,3,"City of Music, National Superior Conservatory ...",7470,Christian de Portzamparc,1987,Paint and colored pencil on print,"16 x 11 3/4"" (40.6 x 29.8 cm)",1995-01-17,Gift of the architect in honor of Lily Auchinc...,Y,...,Architecture,1.1995,NaN,NaN,40.6401,NaN,29.8451,NaN,NaN,NaN
2,4,"Villa near Vienna Project, Outside Vienna, Aus...",7605,Emil Hoppe,1903,"Graphite, pen, color pencil, ink, and gouache ...","13 1/2 x 12 1/2"" (34.3 x 31.8 cm)",1997-01-15,Gift of Jo Carole and Ronald S. Lauder,Y,...,Architecture,1.1997,NaN,NaN,34.3000,NaN,31.8000,NaN,NaN,NaN
3,5,"The Manhattan Transcripts Project, New York, N...",7056,Bernard Tschumi,1980,Photographic reproduction with colored synthet...,"20 x 20"" (50.8 x 50.8 cm)",1995-01-17,Purchase and partial gift of the architect in ...,Y,...,Architecture,2.1995,NaN,NaN,50.8000,NaN,50.8000,NaN,NaN,NaN
4,6,"Villa, project, outside Vienna, Austria, Exter...",7605,Emil Hoppe,1903,"Graphite, color pencil, ink, and gouache on tr...","15 1/8 x 7 1/2"" (38.4 x 19.1 cm)",1997-01-15,Gift of Jo Carole and Ronald S. Lauder,Y,...,Architecture,2.1997,NaN,NaN,38.4000,NaN,19.1000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130257,217983,Seul/NY/MAX,4469,Nam June Paik,None,VHS,None,2008-10-08,The Gilbert and Lila Silverman Fluxus Collecti...,N,...,Media,FC5038,NaN,NaN,0.0000,NaN,0.0000,0.0,NaN,NaN
130258,217984,Fluxus-Manifestatie in en Rond Kunsthandel Monet,None,None,1962,VHS,None,2008-10-08,The Gilbert and Lila Silverman Fluxus Collecti...,N,...,Media,FC5039,NaN,NaN,0.0000,NaN,0.0000,0.0,NaN,NaN
130259,217985,Fluxphone Compositions,67695,Ely Ramen,1969,Cassette,None,2008-10-08,The Gilbert and Lila Silverman Fluxus Collecti...,N,...,Media,FC5040,NaN,NaN,0.0000,NaN,0.0000,0.0,NaN,NaN
130260,217986,Unidentified,None,None,None,Cassette,None,2008-10-08,The Gilbert and Lila Silverman Fluxus Collecti...,N,...,Media,FC5041,NaN,NaN,0.0000,NaN,0.0000,0.0,NaN,NaN


Use info to see overall information about artists dataframe\
Observations: 
* Null values for: Nationality, Gender, Birth Year, Death Year
* Column names do not follow conventions
* Birth Year and Death Year are float datatypes

In [19]:
artists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15091 entries, 0 to 15090
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Artist ID    15091 non-null  int64  
 1   Name         15091 non-null  object 
 2   Nationality  12603 non-null  object 
 3   Gender       12019 non-null  object 
 4   Birth Year   11237 non-null  float64
 5   Death Year   4579 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 707.5+ KB


Change column names

In [20]:
artists = clean_columns(artists)

Change birth_year and death_year type to dates

In [21]:
artists.birth_year.unique()
artists.death_year.unique()

array([1992.,   nan, 1966., 1995., 1997., 1964., 2013., 1942., 1941.,
       1940., 2000., 1976., 1949., 2008., 1973., 1953., 1991., 1971.,
       2010., 2011., 2007., 1994., 1967., 1881., 1984., 2002., 1951.,
       1988., 1987., 1978., 1993., 1894., 1999., 1975., 1996., 1969.,
       2012., 1983., 1989., 1959., 2003., 1936., 2006., 1961., 1998.,
       1980., 1982., 1877., 1956., 1879., 1946., 1887., 1985., 2005.,
       1927., 1952., 1871., 1931., 1960., 1948., 2009., 2004., 1965.,
       1955., 2001., 1981., 1990., 1924., 1986., 1968., 1882., 1958.,
       2014., 2016., 1972., 1944., 1905., 1938., 1902., 1934., 1979.,
       1963., 2015., 1954., 1898., 1903., 1950., 1922., 1893., 1910.,
       1925., 1908., 1962., 1957., 1900., 1909., 1977., 1828., 1933.,
       1870., 1876., 1896., 1916., 1930., 1970., 1885., 1947., 1943.,
       1929., 1912., 1919., 1937., 1945., 1939., 1890., 1906., 1926.,
       1856., 1932., 1863., 1974., 1875., 1913., 1892., 1878., 1928.,
       1901., 1920.,

In [22]:
artists['birth_year'] = pd.to_datetime(artists['birth_year'], format='%Y')
artists['death_year'] = pd.to_datetime(artists['death_year'], format='%Y')
artists.head()

,artist_id,name,nationality,gender,birth_year,death_year
0,1,Robert Arneson,American,Male,1930-01-01,1992-01-01
1,2,Doroteo Arnaiz,Spanish,Male,1936-01-01,NaT
2,3,Bill Arnold,American,Male,1941-01-01,NaT
3,4,Charles Arnoldi,American,Male,1946-01-01,NaT
4,5,Per Arnoldi,Danish,Male,1941-01-01,NaT


Check for duplicates

In [ ]:
df_artists.duplicated().sum()

Strip all leading or trailing whitespace

In [23]:
cols = artists.select_dtypes(object).columns
artists[cols] = artists[cols].apply(lambda x: x.str.strip())

Check unique values for nationality and gender

In [24]:
artists.gender.unique()

array(['Male', 'Female', None, 'male'], dtype=object)

In [25]:
artists['gender'].replace(['male'],['Male'], inplace=True)
artists.gender.unique()

array(['Male', 'Female', None], dtype=object)

In [26]:
artists.nationality.unique()

array(['American', 'Spanish', 'Danish', 'Italian', 'French', 'Estonian',
       'Mexican', 'Swedish', None, 'Israeli', 'British', 'Finnish',
       'Polish', 'Japanese', 'Guatemalan', 'Colombian', 'Romanian',
       'Russian', 'German', 'Argentine', 'Kuwaiti', 'Various', 'Belgian',
       'Dutch', 'Norwegian', 'Nationality unknown', 'Chilean', 'Swiss',
       'Costa Rican', 'Czech', 'Brazilian', 'Austrian', 'Canadian',
       'Australian', 'Ukrainian', 'Hungarian', 'Haitian', 'Congolese',
       'Bolivian', 'Cuban', 'Yugoslav', 'Portuguese', 'Indian',
       'Icelandic', 'Irish', 'Guyanese', 'Uruguayan', 'Slovak',
       'Croatian', 'Greek', 'Peruvian', 'Chinese', 'Venezuelan',
       'Turkish', 'Panamanian', 'Algerian', 'Ecuadorian', 'South African',
       'Iranian', 'Korean', 'Canadian Inuit', 'Paraguayan',
       'Luxembourgish', 'Nicaraguan', 'Zimbabwean', 'Moroccan',
       'Tanzanian', 'Bulgarian', 'Tunisian', 'Sudanese', 'Taiwanese',
       'Ethiopian', 'Slovenian', 'Scottish',

Reviewing this will be easier in alphabetical order. Sorted function will not work with nan value present, therefore nan values are replaced with 'Nationality Unknown'.

In [27]:
artists.nationality.fillna('Nationality Unknown', inplace=True)
nat = artists.nationality.unique()
sorted(nat)

['Afghan',
 'Albanian',
 'Algerian',
 'American',
 'Angolan',
 'Argentine',
 'Australian',
 'Austrian',
 'Azerbaijani',
 'Bahamian',
 'Belgian',
 'Bolivian',
 'Bosnian',
 'Brazilian',
 'British',
 'Bulgarian',
 'Burkinabe',
 'Cambodian',
 'Cameroonian',
 'Canadian',
 'Canadian Inuit',
 'Catalan',
 'Chilean',
 'Chinese',
 'Colombian',
 'Congolese',
 'Costa Rican',
 'Croatian',
 'Cuban',
 'Cypriot',
 'Czech',
 'Czechoslovakian',
 'Danish',
 'Dutch',
 'Ecuadorian',
 'Egyptian',
 'Emirati',
 'Estonian',
 'Ethiopian',
 'Filipino',
 'Finnish',
 'French',
 'Georgian',
 'German',
 'Ghanaian',
 'Greek',
 'Guatemalan',
 'Guyanese',
 'Haitian',
 'Hungarian',
 'Icelandic',
 'Indian',
 'Indonesian',
 'Iranian',
 'Iraqi',
 'Irish',
 'Israeli',
 'Italian',
 'Ivorian',
 'Japanese',
 'Kazakhstani',
 'Kenyan',
 'Korean',
 'Kuwaiti',
 'Kyrgyzstani',
 'Latvian',
 'Lebanese',
 'Lithuanian',
 'Luxembourgish',
 'Macedonian',
 'Malaysian',
 'Malian',
 'Mauritanian',
 'Mexican',
 'Moroccan',
 'Mozambican',
 'N

No duplicate countries with different spellings. Three different spellings of 'Nationality Unknown' which need replacing. 

In [28]:
artists['nationality'].replace({'nationality unknown':'Nationality Unknown', 'Nationality unknown':'Nationality Unknown'}, inplace=True)

Check max and min dates for sensible values.

In [29]:
artists.describe()

,artist_id,birth_year,death_year
count,15091.000000,11237,4579
mean,18297.556027,1930-11-08 12:23:46.288155136,1974-04-15 21:00:07.075780736
min,1.000000,1730-01-01 00:00:00,1795-01-01 00:00:00
25%,4195.500000,1910-01-01 00:00:00,1958-01-01 00:00:00
50%,8593.000000,1936-01-01 00:00:00,1980-01-01 00:00:00
75%,33088.500000,1956-01-01 00:00:00,1998-01-01 00:00:00
max,67695.000000,2012-01-01 00:00:00,2017-01-01 00:00:00
std,16632.963898,NaN,NaN


Create calculated column for age to check difference between birth and death is sensible.   


In [30]:
artists['age'] = (artists['death_year'] - artists['birth_year'])

In [33]:
artists.describe()

,artist_id,birth_year,death_year,age
count,15090.000000,11236,4578,4565
mean,18296.230881,1930-11-10 12:38:57.629049344,1974-04-13 21:40:58.191349936,26411 days 16:30:10.646221056
min,1.000000,1730-01-01 00:00:00,1795-01-01 00:00:00,365 days 00:00:00
25%,4195.250000,1910-01-01 00:00:00,1958-01-01 00:00:00,23376 days 00:00:00
50%,8592.500000,1936-01-01 00:00:00,1980-01-01 00:00:00,27393 days 00:00:00
75%,33081.500000,1956-01-01 00:00:00,1998-01-01 00:00:00,30316 days 00:00:00
max,67695.000000,2012-01-01 00:00:00,2017-01-01 00:00:00,39446 days 00:00:00
std,16632.718396,NaN,NaN,5817 days 12:08:19.527092544


Max value for age is 130 years. Oldest person ever alive was 122 so this is not possible. View all rows where age is larger than possible. 

In [31]:
artists[artists['age']>'44530 days']

,artist_id,name,nationality,gender,birth_year,death_year,age
12486,38294,"Union Paper Bag Machine Company, Philadelphia, PA",Nationality Unknown,None,1869-01-01,1999-01-01,47481 days


'Union Paper Bag Machine Company' is not an artist, this row can be dropped.

In [32]:
artists.drop(12486, inplace=True)

Use info to see overall information about artists dataframe\
Observations: 
* Null values for: Artist ID, Name, Date, Medium, Dimensions, Acquisition date, credit, all dimensions
* Column names not using convention
* Artist Id is object - should be int64
* Date and aquisition date are object - should be datetime

In [34]:
artworks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130262 entries, 0 to 130261
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Artwork ID          130262 non-null  int64  
 1   Title               130210 non-null  object 
 2   Artist ID           128802 non-null  object 
 3   Name                128802 non-null  object 
 4   Date                127954 non-null  object 
 5   Medium              118343 non-null  object 
 6   Dimensions          118799 non-null  object 
 7   Acquisition Date    124799 non-null  object 
 8   Credit              127192 non-null  object 
 9   Catalogue           130262 non-null  object 
 10  Department          130262 non-null  object 
 11  Classification      130262 non-null  object 
 12  Object Number       130262 non-null  object 
 13  Diameter (cm)       1399 non-null    float64
 14  Circumference (cm)  10 non-null      float64
 15  Height (cm)         111893 non-nul

Change column names

In [35]:
artworks = clean_columns(artworks)

Change data types for: artist_id, date and aquisition_date

Artist id column is object and will not convert to int due to some rows containing multiple ids. 